In [10]:
import numpy as np
import matplotlib.pyplot as plt
import causallearn
from causallearn.utils.cit import *
from causallearn.graph.GraphClass import CausalGraph
import pandas as pd
from itertools import combinations, permutations
from numpy import ndarray
from typing import Dict, List, Tuple
from causallearn.utils.cit import fisherz
import random
import time

import scipy as sp

# Initialize Dataset (Sample num_traj trajectories of length T)

In [11]:
beta = 0.6
num_states = 50
num_inputs = 20

A = np.random.normal(0,1, (num_states, num_states))
B = np.ones((num_states, num_inputs)) * beta
T = 2
num_traj = 20000
# num_traj = 10000

for traj_id in range(num_traj):
    x_vec = np.zeros((num_states, T))
    x_vec[:, 0] = np.random.randn(num_states)
    u_vec = np.random.randn(num_inputs, T)
    #w_vec = np.random.randn(2,T) * np.sqrt(1 - alpha**2 - beta**2)
    
    for t in range(T-1):
    #     print(t)
        x_vec[:, t+1] = A @ x_vec[:, t] + B @ u_vec[:,t] #+ w_vec[:, t]
    
    x_u_vec = np.block([[x_vec], [u_vec]])
    
    if traj_id == 0:
        traj_dataset = np.zeros((num_traj, x_u_vec.shape[0], x_u_vec.shape[1]))
        traj_dataset[0, :, :] = x_u_vec
    else:
        traj_dataset[traj_id, :, :] = x_u_vec


In [12]:
# plt.plot(traj_dataset)
traj_dataset.shape

(20000, 70, 2)

In [13]:
assert traj_dataset.shape == (num_traj, num_states + num_inputs, T)

In [14]:
num_traj, num_quantities, _ = traj_dataset.shape
# num_quantities

# Modified PC Algorithm for Learning Number of Inputs

In [35]:
def correlation_test(data, m, rho_m):
    num_traj, num_quantities, _ = data.shape
    input_indices = []
    
    for i in range(num_quantities):
        node_indices_to_test = np.random.choice(num_quantities, m)
        for count, j in enumerate(node_indices_to_test):
            corr = sp.stats.pearsonr(traj_dataset[:, j, 0], traj_dataset[:, i, 1])[0]
            if corr >= rho_m:
                break
            if count == m-1:
                input_indices.append(i)
    
    return input_indices

def compute_rhos(A, B):
    # Returns num_states array x (num_states + num_inputs)
    # Accurate only under the given assumptions.
    
    variances = np.linalg.norm(np.block([A, B]), axis = 1)
    rho_array = np.block([A, B]) / variances[:,None]
    
    return rho_array

In [36]:
rho_vec = np.sort(abs(compute_rhos(A, B).flatten()), axis=None)
rho_vec[0:350]

array([5.03732466e-05, 1.09017900e-04, 1.48798718e-04, 4.04493533e-04,
       4.35406843e-04, 4.46852600e-04, 4.63873653e-04, 4.65179085e-04,
       4.78361230e-04, 5.64579597e-04, 6.47314471e-04, 7.32900338e-04,
       9.25030194e-04, 9.29905475e-04, 1.11487064e-03, 1.30602607e-03,
       1.37881636e-03, 1.38697422e-03, 1.41100224e-03, 1.46242090e-03,
       1.48033994e-03, 1.60737815e-03, 1.65011101e-03, 1.68227463e-03,
       1.69590402e-03, 1.74346538e-03, 1.84300974e-03, 1.96055780e-03,
       1.96302051e-03, 1.99675346e-03, 1.99919980e-03, 2.14395217e-03,
       2.20491383e-03, 2.20518863e-03, 2.27603194e-03, 2.31595983e-03,
       2.33141228e-03, 2.47477235e-03, 2.48784458e-03, 2.65541887e-03,
       2.68468573e-03, 2.71335430e-03, 2.75367424e-03, 2.82493895e-03,
       2.86706213e-03, 2.95016919e-03, 3.00978727e-03, 3.06283347e-03,
       3.07961796e-03, 3.12057023e-03, 3.20091589e-03, 3.22659762e-03,
       3.23765571e-03, 3.27731033e-03, 3.35232346e-03, 3.36180074e-03,
      

In [37]:
list_1 = [3, 4, 5]
list_2 = [4, 5, 6]

list_1_not_2 = [k for k in list_1 if k not in list_2]
list_2_not_1 = [k for k in list_2 if k not in list_1]

In [38]:
begin_time = time.time()

m_list = [2*x + 2 for x in list(range(10))]
rho_min_list = [x * 0.02 + 0.02 for x in list(range(5))]
input_indices_true = list(range(num_states, num_states + num_inputs))
num_trials = 10

false_positive_avg_array = np.zeros((len(m_list), len(rho_min_list)))
missed_detection_avg_array = np.zeros((len(m_list), len(rho_min_list)))

for count_m, m in enumerate(m_list):
    print("\n")
    print("count_m:", count_m)
    for count_rho, rho_min in enumerate(rho_min_list):
        print("count_rho", count_rho)
        false_positive_quantity_list = []
        missed_detection_quantity_list = []
        for trial_id in range(num_trials):
            input_indices_est = correlation_test(traj_dataset, m, rho_min)
            false_positive_quantity_list.append(len([k for k in input_indices_est if k not in input_indices_true]))
            missed_detection_quantity_list.append(len([k for k in input_indices_true if k not in input_indices_est]))
        false_positive_avg_array[count_m, count_rho] = sum(false_positive_quantity_list)/len(false_positive_quantity_list)
        missed_detection_avg_array[count_m, count_rho] = sum(missed_detection_quantity_list)/len(missed_detection_quantity_list)

end_time = time.time()
print("Time:", end_time - begin_time)
        
# print(correlation_test(traj_dataset, 10, 0.05))





count_m: 0
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 1
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 2
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 3
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 4
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 5
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 6
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 7
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 8
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4


count_m: 9
count_rho 0
count_rho 1
count_rho 2
count_rho 3
count_rho 4
Time: 122.21105098724365


In [39]:
print(false_positive_avg_array)
print("\n")
print(missed_detection_avg_array)

[[ 8.  10.4 12.2 20.2 34. ]
 [ 1.   2.2  2.9  9.8 26.3]
 [ 0.2  0.5  0.7  4.1 16. ]
 [ 0.1  0.   0.1  3.1 12.2]
 [ 0.   0.1  0.2  1.9  9.4]
 [ 0.   0.   0.   1.4  6.6]
 [ 0.   0.   0.   0.8  4.1]
 [ 0.   0.   0.   0.4  4. ]
 [ 0.   0.   0.   0.3  1.8]
 [ 0.   0.   0.   0.1  1.6]]


[[0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.1 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.1 0.  0.  0.  0. ]
 [0.4 0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0. ]]


In [245]:
# Save results:

path_directory = "/Users/chih-yuanchiu/Desktop/Code/Causality_Time_Series/data/"

path_file = path_directory + "false_positive_avg_array.csv"
false_positive_avg_array_pd = pd.DataFrame(false_positive_avg_array)
false_positive_avg_array_pd.to_csv(path_file, index=False)

path_file = path_directory + "missed_detection_avg_array.csv"
missed_detection_avg_array_pd = pd.DataFrame(missed_detection_avg_array)
missed_detection_avg_array_pd.to_csv(path_file, index=False)



# Kshitij's algorithms:

In [ ]:
def compute_sample_corr(data,i,j, num_samples):
    avg_t_plus = (1/num_samples)*data[:,i].sum()
    avg_t = (1/num_samples)*data[:,j].sum()
    diff_t_plus = data[:, i] - avg_t_plus
    diff_t = data[:,j] - avg_t_plus
    diff_t_plus_sq = (diff_t_plus**2).sum()
    diff_t_sq = (diff_t**2).sum()
    corr = (np.dot(diff_t_plus, diff_t))/np.sqrt((diff_t_plus_sq*diff_t_sq))
    return corr

def modified_pc(data, m, num_samples, num_observations, rho_m):
    input_set = []
    for i in range(data.shape[1]):
        subset = np.array(random.sample(list(range(data.shape[1])), m))
        for j in subset:
            rho_m_hat = compute_sample_corr(data, i,j, num_samples)
            if abs(rho_m_hat) >= 0.5*rho_m:
                break 
            if j == subset[-1]:
                input_set.append(i)
    return input_set


